# Stock data analysis
#### part1 : Three ways to import stock data

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div> 

<div style="text-align: right"> Initial upload: 2020.07.21 </div> 
<div style="text-align: right"> Last update: 2020.07.21 </div> 

주식 데이터 분석을 위해서 가장 먼저 데이터를 읽어와야 한다.  
네이버증권 크롤링, FinanceDataReader, pandas_datareader/ Yahoo finance 라이브러리를 사용한 3가지 방법에 대하여 소개한다.

### 0. 한국거래소(KRX)에서 종목코드 가져오기  
분석하기 전에 KRX에 기록된 상장기업명과 종목코드(6자리)를 저장해둔다. 종목검색에 활용할 수있다. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings; warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

먼저 KRX 홈페이지(http://kind.krx.co.kr/)를 방문하여 `장법인상세정보`/ `장법인목록` 화면으로 이동한다.  
(또는 [[다음]](http://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage) 링크를 클릭한다.)

<img src = './pic/1.1.jpg' width = 600>

오른쪽 아래의 `EXCEL` 버튼을 클릭하면 상장법인 목록 파일을 다운로드 할 수있다.  
여기서는 `pandas`의 `read_html`을 활용하여 정보를 데이터프레임으로 읽어들인다.  
`read_html()`은 HTML에서 'table' '/table'태그를 찾아 자동으로 DataFrame형식으로 만들어준다.

In [2]:
stock_code = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', 
                          header = 0)[0]
# 주의 : 마지막에 [0]을 반드시 추가해야함. read_html은 table들을 읽어 리스트로 저장함

In [3]:
stock_code.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
2,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"권순호, 정경구",http://www.hdc-dvp.com,서울특별시
4,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시


국내 주식시장의 종목코드는 6자리 이다. 예를 들어 `GS글로벌`의 종목코드는 `001250`이다.  
그런데 저장된 종목코드는 앞자리 0이 생략되어 있다. `map`함수로 6자리를 완성한다.

In [4]:
stock_code['종목코드'] = stock_code['종목코드'].map('{:06d}'.format)
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌

분석에 필요한 컬럼만 추출하고, 컬럼명을 영어로 수정한다. 

In [5]:
stock_code = stock_code[['회사명', '종목코드', '업종', '주요제품']]
stock_code = stock_code.rename(columns = {'회사명':'name', '종목코드':'code', '업종':'sectors',
                                         '주요제품': 'products'})

In [6]:
stock_code.head()

,name,code,sectors,products
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프
1,GS,078930,기타 금융업,지주회사/부동산 임대
2,GS글로벌,001250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서..."
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등"
4,LG이노텍,011070,전자부품 제조업,기타 전자부품 제조업


In [7]:
stock_code.shape

(2353, 4)

데이터를 `csv` 확장자 파일로 저장한다.

In [8]:
stock_code.to_csv('data/stock_code.csv', index=False)

In [9]:
pd.read_csv('data/stock_code.csv').head()

,name,code,sectors,products
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프
1,GS,78930,기타 금융업,지주회사/부동산 임대
2,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서..."
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등"
4,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업


### 1. 네이버금융에서 종목 데이터 가져오기  
- 참고자료 : https://excelsior-cjh.tistory.com/109

네이버금융(finance.naver.com)에 접속하여 `삼성전자`를 검색해본다.

<img src ='./pic/1.2.jpg' width = 600>

아래 쪽에 `시세` 버튼을 클릭한다.

<img src ='./pic/1.3.jpg' width = 600>

페이지 아래쪽에 일별 시세가 표시된다.  
그리고 이 페이지의 주소는  
`https://finance.naver.com/item/sise.nhn?code=` `종목코드` 로 구성되어 있다.  
예를들어 삼성전자의 시세 페이지 주소는  
`https://finance.naver.com/item/sise.nhn?code=005930` 이다.   
<img src ='./pic/1.4.jpg' width = 600>

이 페이지에서 시세정보는 시간대별 시세, 일별 시세 두가지가 있다.  
우리는 일별 시세에 관심이 있다.  
소스를 살펴보면 일별 시세 정보는 `http://finance.naver.com/item/sise_day.nhn?code=005930`에 있는 것을 확인 할 수가 있다.  
그리고 이 일별 시세 정보는 html에서 `table`로 정의되어 있으므로 `pandas`의 `read_html`을 사용하여 읽어올 수가 있다.  
<img src ='./pic/1.6.jpg' width = 1000>
<img src ='./pic/1.5.jpg' width = 1000>

예를들어 시세의 첫번째 페이지 주소는 `http://finance.naver.com/item/sise_day.nhn?code=005930&page=1`이다.  

먼저 원하는 종목의 시제정보 주소를 리턴하는 함수를 만든다.

In [10]:
def get_url(item_name, stock_code): 
    code = stock_code.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    url = 'http://finance.naver.com/item/sise_day.nhn?code={}'.format(code)
    url = url.replace(" ", "") #code 뒤에 공백이 생겨, 함수를 추가함
    print("요청 URL = {}".format(url)) 
    return url

`삼성전자`의 시세정보 페이지 주소를 리턴하는 함수를 시험해본다.

In [11]:
item_name = '삼성전자'
url = get_url(item_name, stock_code)

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=005930


이제 `read_html()`을 사용하여 일별 시세 정보를 읽어온다.  
여기서는 1 ~ 20 페이지 까지의 정보를 읽어 오도록 한다.

In [12]:
stock_data = pd.DataFrame()
for page in range(1, 21): # 1 ~ 20페이지만 가져오기
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    temp = pd.read_html(pg_url, header=0)[0]
    stock_data = stock_data.append(temp, ignore_index=True)

In [13]:
stock_data.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.07.21,55300.0,1100.0,55200.0,55400.0,54800.0,18194959.0
2,2020.07.20,54200.0,200.0,54800.0,54800.0,54000.0,10507530.0
3,2020.07.17,54400.0,600.0,54200.0,54700.0,54100.0,10096174.0
4,2020.07.16,53800.0,900.0,54800.0,54800.0,53800.0,16779127.0


In [14]:
stock_data.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
295,2019.10.07,47750.0,250.0,48350.0,48700.0,47650.0,7047273.0
296,2019.10.04,48000.0,400.0,47400.0,48650.0,47350.0,9331695.0
297,2019.10.02,47600.0,1250.0,48350.0,48400.0,47600.0,8382463.0
298,2019.10.01,48850.0,200.0,48900.0,49100.0,48650.0,6206035.0
299,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 페이지당 (15*페이지수-2)일치의 정보가 저장된다.  
- 위의 예에서는 15*20페이지-2 = 298일치의 정보가 저장되었다.

컬럼명과 데이터 타입을 변경한다.

In [15]:
# 한글로 된 컬럼명을 영어로 바꿔줌 
stock_data = stock_data.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', 
        '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 

# 데이터의 타입을 int형으로 바꿔줌 
stock_data[['close', 'diff', 'open', 'high', 'low', 'volume']] = stock_data[['close', 'diff', 
                            'open', 'high', 'low', 'volume']].astype('float64')

In [16]:
stock_data.head()

,date,close,diff,open,high,low,volume
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.07.21,55300.0,1100.0,55200.0,55400.0,54800.0,18194959.0
2,2020.07.20,54200.0,200.0,54800.0,54800.0,54000.0,10507530.0
3,2020.07.17,54400.0,600.0,54200.0,54700.0,54100.0,10096174.0
4,2020.07.16,53800.0,900.0,54800.0,54800.0,53800.0,16779127.0


`date` 컬럼을 datetime 타입으로 변경한다.

In [17]:
stock_data['date'] = pd.to_datetime(stock_data['date'])

In [18]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    200 non-null    datetime64[ns]
 1   close   200 non-null    float64       
 2   diff    200 non-null    float64       
 3   open    200 non-null    float64       
 4   high    200 non-null    float64       
 5   low     200 non-null    float64       
 6   volume  200 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 16.5 KB


결측치를 제거한다.

In [19]:
stock_data = stock_data.dropna(how = 'any')

In [20]:
stock_data.head()

,date,close,diff,open,high,low,volume
1,2020-07-21,55300.0,1100.0,55200.0,55400.0,54800.0,18194959.0
2,2020-07-20,54200.0,200.0,54800.0,54800.0,54000.0,10507530.0
3,2020-07-17,54400.0,600.0,54200.0,54700.0,54100.0,10096174.0
4,2020-07-16,53800.0,900.0,54800.0,54800.0,53800.0,16779127.0
5,2020-07-15,54700.0,900.0,54400.0,55000.0,54300.0,24051450.0


데이터를 날짜순으로 정렬한다.

In [21]:
stock_data = stock_data.sort_values(by = 'date')

In [22]:
stock_data.head()

,date,close,diff,open,high,low,volume
298,2019-10-01,48850.0,200.0,48900.0,49100.0,48650.0,6206035.0
297,2019-10-02,47600.0,1250.0,48350.0,48400.0,47600.0,8382463.0
296,2019-10-04,48000.0,400.0,47400.0,48650.0,47350.0,9331695.0
295,2019-10-07,47750.0,250.0,48350.0,48700.0,47650.0,7047273.0
294,2019-10-08,48900.0,1150.0,47900.0,49000.0,47600.0,14239367.0


`date` 컬럼을 인덱스로 지정한다.

In [23]:
stock_data = stock_data.set_index(keys = 'date', drop = True)

In [24]:
stock_data.head()

,close,diff,open,high,low,volume
date,,,,,,
2019-10-01,48850.0,200.0,48900.0,49100.0,48650.0,6206035.0
2019-10-02,47600.0,1250.0,48350.0,48400.0,47600.0,8382463.0
2019-10-04,48000.0,400.0,47400.0,48650.0,47350.0,9331695.0
2019-10-07,47750.0,250.0,48350.0,48700.0,47650.0,7047273.0
2019-10-08,48900.0,1150.0,47900.0,49000.0,47600.0,14239367.0


### 2. FinancesDateReader 사용하기  
- 참고자료 : [[FinanceDataReader tutorial]](https://github.com/FinanceData/FinanceDataReader/blob/master/tutorial/FinanceDataReader%20Tutorial%20-%20%EC%97%AC%EB%9F%AC%20%EC%A2%85%EB%AA%A9%EC%9D%98%20%EA%B0%80%EA%B2%A9%EC%9D%84%20%ED%95%9C%EB%B2%88%EC%97%90.ipynb)

In [25]:
# 라이브러리 설치
# !pip -q install finance-datareader

In [26]:
import FinanceDataReader as fdr

FinanceDataReader 라이브러리로 주식 데이터를 받기 위해서는 종목코드를 인자로 입력해야 한다.  
따라서 `종목명`을 입력하면 `종목 코드`를 리턴하는 함수 `get_code()`를 생성한다. 

In [27]:
def get_code(item_name, stock_code): 
    code = stock_code.loc[stock_code['name'] == item_name, 'code'] 
    return code.values[0]

In [28]:
print(get_code('삼성전자', stock_code))

005930


In [29]:
item_code = get_code('삼성전자', stock_code)
stock_data = fdr.DataReader(item_code, '2019-10-01', '2020-7-21')

In [30]:
stock_data.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2019-10-01,48900,49100,48650,48850,6206035,-0.004077
2019-10-02,48350,48400,47600,47600,8382463,-0.025589
2019-10-04,47400,48650,47350,48000,9331695,0.008403
2019-10-07,48350,48700,47650,47750,7047273,-0.005208
2019-10-08,47900,49000,47600,48900,14239367,0.024084


In [31]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 200 entries, 2019-10-01 to 2020-07-21
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    200 non-null    int64  
 1   High    200 non-null    int64  
 2   Low     200 non-null    int64  
 3   Close   200 non-null    int64  
 4   Volume  200 non-null    int64  
 5   Change  200 non-null    float64
dtypes: float64(1), int64(5)
memory usage: 10.9 KB


### 3. Yahoo finance 사용하기

- **작성중**